<a href="https://colab.research.google.com/github/SergioPGJunior/detecthypotension/blob/master/grid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primeiramente, é necessário excutar o comando abaixo, pois o Colab não possui o pacote wfdb instalado.

In [ ]:
! pip install wfdb

É realizado o import dos pacotes necessários para a organização da base de dados.

In [ ]:
import pandas as pd
import numpy as np
import wfdb

Em seguida, é criada uma função para carregar os sinais de ECG da base e outra função para organizar as matrizes com as amostras e os labels.

In [ ]:
def load_ecg(file):
    # Leitura e armazenamento dos dados
    record = wfdb.rdrecord(file)

    # Leitura e armazenamento das anotações
    annotation = wfdb.rdann(file, "atr")

    #Extrai os sinais
    p_signal = record.p_signal

    #Verifica se a frequência é 360
    assert record.fs == 360, "Frequência de amostragem não é 360"

    #Extrai os símbolos e anotações
    atr_sym = annotation.symbol
    atr_sample = annotation.sample

    return p_signal, atr_sym, atr_sample

In [ ]:
def build_XY(p_signal, df_ann, num_cols, abnormal, num_sec, fs):
  # this function builds the X,Y matrices for each beat
  # it also returns the original symbols for Y

  num_rows = len(df_ann)
  X = np.zeros((num_rows, num_cols))
  Y = np.zeros((num_rows, 1))
  sym = []

  # keep track of rows
  max_row = 0
  for atr_sample, atr_sym in zip(df_ann.atr_sample.values, df_ann.atr_sym.values):
      left = max([0, (atr_sample - num_sec * fs)])
      right = min([len(p_signal), (atr_sample + num_sec * fs)])
      x = p_signal[left: right]
      if len(x) == num_cols:
          X[max_row, :] = x
          Y[max_row, :] = int(atr_sym in abnormal)
          sym.append(atr_sym)
          max_row += 1
  X = X[:max_row, :]
  Y = Y[:max_row, :]
  return X, Y, sym

Com o auxílio das funções criadas anteriormente, os sinais da base são carregados e as matrizes criadas.

In [ ]:
# Caminho onde os arquivos da base de dados estão armazenados
path = "/content/drive/My Drive/mit-bih-arrhythmia-database-1.0.0/"

# Cria um dataframe com os nomes dos arquivos
rec = pd.read_csv(path + "RECORDS", names=['n'], dtype=str)

# Lista de nonbeats e abnormal
nonbeat = ['[', '!', ']', 'x', '(', ')', 'p', 't', '`', 'u', '\\', '^', '|', '~', '+', 's', 'T', '*', 'D', '=', '"', '@', 'Q', '?']
abnormal = ['V', 'A']

num_sec = 1
fs = 360

#Inicilaiza os vetores
num_cols = 2 * num_sec * fs #numero de colunas que vai armazenar um amostra 
X_all = np.zeros((1, num_cols)) #Cria um vetor com numero de elementos igual a num_cols e preenche com zeros 
Y_all = np.zeros((1, 1)) #cria um vetor com com um elemento e preenche com zero
sym_all = []

#Lista para controlar o número de batimentos de cada paciente
max_rows = []

for pt in rec.n:
  file = path + pt
  p_signal, atr_sym, atr_sample = load_ecg(file)
  
  if 'A' and 'V' in atr_sym:
    #Seleciona o sinal MLII
    p_signal = p_signal[:, 0]
    
    # Cria um df para excluir o que mão é um batimento
    df_ann = pd.DataFrame({'atr_sym': atr_sym,
                          'atr_sample': atr_sample})
    df_ann = df_ann.loc[df_ann.atr_sym.isin(abnormal + ['N'])]
   
    X, Y, sym = build_XY(p_signal, df_ann, num_cols, abnormal, num_sec, fs)
    sym_all = sym_all + sym
    max_rows.append(X.shape[0])
    X_all = np.append(X_all, X, axis=0)
    Y_all = np.append(Y_all, Y, axis=0)

#Exclui a primeira linha que é formada por zeros
X_all = X_all[1:, :]
Y_all = Y_all[1:, :]

#Verifica se os tamanhos das matrizes fazem sentido
assert np.sum(max_rows) == X_all.shape[0], 'number of X, max_rows rows messed up'
assert Y_all.shape[0] == X_all.shape[0], 'number of X, Y rows messed up'
assert Y_all.shape[0] == len(sym_all), 'number of Y, sym rows messed up'

d = {x:np.count_nonzero(Y_all == x) for x in [0,1]} #Calcula quantas amostras cada classe possui 0 -> Normal, 1 -> Arritmia

print(d)
print(len(X_all)) #Quantidade total de amostras

Com o objetivo de procuar os melhores valores dos hiper-parâmetros número de neurônios e taxa de aprendizagem, é executada a busca em grade com alguns valores para esses parâmetros. Como foi utilizado um modelo de rede neural do keras e a função de busca em grade é do SkLearn, foi necessário utilizar a função KerasClassifier para "envelopar" a rede neural com o padrão do SKLearn.

In [ ]:
import tensorflow as tf
import keras
from keras import layers
from keras import models
from keras import utils
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Activation
from keras.optimizers import SGD
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(learn_rate=0.01, neurons=3):
  model = Sequential()
  model.add(Dense(neurons, input_dim=X_all.shape[1], activation='relu'))
  model.add(Dropout(rate=0.1))
  model.add(Dense(1, activation='sigmoid'))

  # Compile model
  opt = SGD(learning_rate=learn_rate)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  
  return model

model_CV = KerasClassifier(build_fn=create_model,  epochs=100, verbose=0)

In [ ]:
learn_rate = [0.001,0.01,0.1]
neurons = [3, 5, 10, 15, 20, 25, 30, 35, 40]

param_grid = dict(neurons=neurons, learn_rate=learn_rate, batch_size=batch_size)
grid = GridSearchCV(estimator=model_CV, 
                    param_grid=param_grid, n_jobs=-1,
                    cv=3)

grid_result = grid.fit(X_all, Y_all)

In [ ]:
busca_grade = pd.DataFrame(grid_result.cv_results_)
busca_grade.to_csv("busca_grade.csv",index=False)